In [6]:
#Here I import the pandas, regular expression and Levensthein libraries
import pandas as pd
import re
import Levenshtein

In [9]:
#Load the metaClean data into a dataframe
df = pd.read_excel("C:/Users/olavr/Documents/Studie/MDDB/DDB management/metaClean43Brightspace.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/olavr/Documents/Studie/MDDB/DDB management/metaClean43Brightspace.xlsx'

In [3]:
#Load the Sales data into a dataframe
df2 = pd.read_excel("C:/Users/olavr/Documents/Studie/MDDB/DDB management/sales.xlsx")

,year,release_date,title,genre,international_box_office,domestic_box_office,worldwide_box_office,production_budget,Unnamed: 8,opening_weekend,theatre_count,avg run per theatre,runtime,keywords,creative_type,url
0,2000,January 1st,Bakha Satang,Drama,76576.0,NaN,76576.0,NaN,NaN,NaN,NaN,NaN,129.0,NaN,Contemporary Fiction,https://www.the-numbers.com/movie/Bakha-Satang...
1,2001,January 12th,Antitrust,Thriller/Suspense,6900000.0,10965209.0,17865209.0,30000000.0,NaN,5486209.0,2433.0,3.1,NaN,NaN,Contemporary Fiction,https://www.the-numbers.com/movie/Antitrust
2,2000,January 28th,Santitos,NaN,NaN,378562.0,NaN,NaN,NaN,NaN,NaN,NaN,105.0,NaN,NaN,https://www.the-numbers.com/movie/Santitos
3,2002,2002 (Wide) by,Frank McKlusky C.I.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.the-numbers.com/movie/Frank-McKlus...
4,2002,January 25th,A Walk to Remember,Drama,4833792.0,41227069.0,46060861.0,11000000.0,NaN,12177488.0,2411.0,5.3,NaN,Coming of Age,Contemporary Fiction,https://www.the-numbers.com/movie/Walk-to-Reme...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30607,2021,January 1st,Jokbeoldu sinmun iyagi,Documentary,12356.0,NaN,12356.0,NaN,NaN,NaN,NaN,NaN,168.0,NaN,Factual,https://www.the-numbers.com/movie/Jokbeoldu-si...
30608,2021,March 5th,My Salinger Year,Drama,914119.0,54730.0,968849.0,NaN,NaN,28851.0,123.0,2.0,101.0,Set in New York City,Contemporary Fiction,https://www.the-numbers.com/movie/My-Salinger-...
30609,2021,January 1st,Escort Vehicle 36,Action,240000.0,NaN,240000.0,NaN,NaN,NaN,NaN,NaN,85.0,NaN,Historical Fiction,https://www.the-numbers.com/movie/Escort-Vehic...
30610,2021,May 21st,The Dry,Thriller/Suspense,16987526.0,364397.0,17351923.0,NaN,NaN,119364.0,186.0,2.5,118.0,Crime Thriller,Contemporary Fiction,https://www.the-numbers.com/movie/Dry-The-(Aus...


In [4]:
#Create matchable parts out of the movie URLs from both dataframes
df2['newUrl'] = df2['url'].str.split("/", expand = True)[:][4]
df['newUrl'] = df['url'].str.split("/", expand = True)[:][4]
df['year'] = df['RelDate'].dt.year


In [5]:
#I convert the string in newUrl to lowercase
df['newUrl'] = df['newUrl'].str.lower()
df2['newUrl'] = df2['newUrl'].str.lower()

In [6]:
#I store the newUrl string and the year in new dataframes to merge these into a new dataframe
df3 = df[['newUrl', 'year']]
df4 = df2[['newUrl', 'year']]

df5 = pd.merge(df3, df4, how='outer')

In [7]:
#Removing the year in parentheses from all movies in the column.

#The regular expression for a four digit string enclosed in parentheses is stored in the pattern variable 
pattern = r'\(\d{4}\)'  

#Here the regular expression sub function is applied to all values in the newUrl column. It checks if there is a year enclosed in parentheses and if there is it removes them.
df5['newUrl'] = df5['newUrl'].apply(lambda x: re.sub(pattern, '', x))

In [8]:
#Removing hyphen from every movie title in the newUrl column to deduplicate further
df5['newUrl'] = df5['newUrl'].apply(lambda x: re.sub("-", '', x))

In [9]:
#I drop the duplicated values
df5 = df5.drop_duplicates()

In [10]:
#I have two loops iterating over all the values in newUrl and year.
#It compares every value in the newUrl column with every other value in this column.
#If the titles arent the same but have a Levenshtein distance of less than two AND the year of release is the same it assumes it is the same movie.
#The values from newUrl are stored in the MovieNames dictionary where the keys are the movie titles from newUrl and the value is the movie title which will be turned into a movie_ID
#The result gives some unique Keys the same value.
movieNames = {}

for titleX, yearX in zip(df5['newUrl'], df5['year']):
    for titleY, yearY in zip(df5['newUrl'], df5['year']):
        if titleX != titleY and yearX == yearY and Levenshtein.distance(titleX, titleY, weights=[1, 1, 3]) <= 2:
            if titleX not in movieNames:
                movieNames[titleX] = titleY
        elif titleX  == titleY:
            if titleX not in movieNames:
                movieNames[titleX] = titleY

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\olavr\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\olavr\AppData\Local\Temp\ipykernel_7668\3083588684.py", line -1, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\olavr\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\olavr\anaconda3\Lib\site-packages\IPython\core\ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\olavr\anaconda3\Lib\site-packages\IPython\core\ultratb.py", line 1199, in structured_traceback
    return VerboseTB.structured_traceback(
           ^

In [ ]:
#The MovieNames dictionary is stored in a new DataFrame with the column names ShortenedURL and DeduplicatedMovieNames
df8 = pd.DataFrame(list(movieNames.items()), columns=['ShortenedURL', 'DeduplicatedMovieNames'])

In [ ]:
#Here I assign a movie ID to every DeduplicatedMovieName and sort the dataframe by movieID
df8['movieID'] = df8.groupby(['DeduplicatedMovieNames']).ngroup() + 1
df8 = df8.sort_values(by='movieID')


In [ ]:
#Removing the DeduplicatedMovieNames column
df8 = df8.drop(columns=['DeduplicatedMovieNames'])

In [ ]:
#Save the Dataframe with the movie name and the MovieID to a .csv file for future use in the database
df8.to_csv('movieID.csv', index= False)